_Note: This is a SQL scoped notebook, and therefore should be attached to a Serverless SQL warehouse for execution._  

In [0]:
DECLARE OR REPLACE VARIABLE catalog_use STRING DEFAULT 'fhir_workshop';
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
SET VARIABLE catalog_use = :catalog_use;
SET VARIABLE schema_use = :schema_use;

In [0]:
DECLARE OR REPLACE VARIABLE volume_path STRING DEFAULT "/Volumes/" || catalog_use || "/" || schema_use || "/landing/";

In [0]:
SELECT
  catalog_use
  ,schema_use
  ,volume_path;

In [0]:
USE IDENTIFIER(catalog_use || "." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
SHOW VOLUMES;

In [0]:
DECLARE OR REPLACE VARIABLE list_stmnt STRING;

SET VARIABLE list_stmnt = "LIST '" || volume_path || "';";

SELECT list_stmnt;

In [0]:
EXECUTE IMMEDIATE list_stmnt;

In [0]:
DROP TABLE IF EXISTS fhir_bronze;

In [0]:
DECLARE OR REPLACE VARIABLE create_streaming_bronze_stmnt STRING;

SET VARIABLE create_streaming_bronze_stmnt = "
CREATE OR REFRESH STREAMING TABLE fhir_bronze (
  file_metadata STRUCT<
    file_path: STRING,
    file_name: STRING,
    file_size: BIGINT,
    file_block_start: BIGINT,
    file_block_length: BIGINT,
    file_modification_time: TIMESTAMP
  > NOT NULL COMMENT 'Original meta date of the file ingested from the volume.'
  ,ingest_time TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP() COMMENT 'The date timestamp the file was ingested.'
  ,bundle_uuid STRING NOT NULL COMMENT 'Unique identifier for the FHIR bundle.'
  ,value STRING COMMENT 'Original JSON record ingested from the volume as a full text string value.'
)
COMMENT 'Ingest FHIR JSON records as Full Text STRING'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true',
  'delta.enableDeletionVectors' = 'true',
  'delta.enableRowTracking' = 'true',
  'quality' = 'bronze'
)
AS SELECT
  _metadata as file_metadata
  ,uuid() as bundle_uuid
  ,* 
FROM STREAM read_files(
  '" || volume_path || "'
  ,format => 'text'
  ,wholeText => true
)
";

SELECT create_streaming_bronze_stmnt;

In [0]:
EXECUTE IMMEDIATE create_streaming_bronze_stmnt;

In [0]:
SELECT * FROM fhir_bronze limit 1;

In [0]:
DROP TABLE IF EXISTS fhir_bronze_variant;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_bronze_variant (
  bundle_uuid STRING NOT NULL COMMENT 'Unique identifier for the FHIR bundle.'
  ,ingest_time TIMESTAMP NOT NULL COMMENT 'The date timestamp the file was ingested.'
  ,file_metadata STRUCT<
    file_path: STRING,
    file_name: STRING,
    file_size: BIGINT,
    file_block_start: BIGINT,
    file_block_length: BIGINT,
    file_modification_time: TIMESTAMP
  > NOT NULL COMMENT 'Original meta date of the file ingested from the volume.'
  ,fhir VARIANT COMMENT 'Original JSON record fully parsed as a variant data type.'
)
COMMENT 'Evaluate FHIR JSON records as VARIANT'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true' 
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,ingest_time
  ,file_metadata
  ,try_parse_json(value) as fhir 
FROM STREAM fhir_bronze;

In [0]:
select * from fhir_bronze_variant limit 1;

In [0]:
select 
  fhir:Meta.EventType::string as EventType
  ,count(*)
from 
  fhir_bronze_variant
group by
  EventType